In [2]:
import cv2
import mediapipe as mp

In [1]:
# Initialize MediaPipe Pose solution.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize MediaPipe drawing utility.
mp_drawing = mp.solutions.drawing_utils

# Capture video from the default camera.
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Convert the BGR image to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and detect the pose.
    results = pose.process(image)

    print(results.x, results.y, results.z)

    """
    results object contains the x y and z axes values of the point that will be drawn
                                visibility number which indicated whether that point is clear
    """


    # Draw the pose annotations on the image.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the resulting image.
    cv2.imshow('MediaPipe Pose', image)

    # Break the loop if 'q' is pressed.
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

pose.close()
cap.release()


NameError: name 'mp' is not defined

In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

def create_sequences(image_folder, label_folder, sequence_length):
    sequences = []
    labels = []

    image_files = sorted(os.listdir(image_folder))
    for i in range(len(image_files) - sequence_length + 1):
        sequence = []
        sequence_labels = []

        for j in range(i, i + sequence_length):
            img_path = os.path.join(image_folder, image_files[j])
            frame = cv2.imread(img_path)
            if frame is None:
                print(f"Failed to read image: {img_path}")
                break
            pose_data = extract_pose_data(frame)
            sequence.append(pose_data)

            label_path = os.path.join(label_folder, image_files[j].replace('.png', '.txt'))
            with open(label_path, 'r') as file:
                if file is None:
                    print(f"failed to read label: {label_path}")
                fall_label = int(file.read().split()[0])
                sequence_labels.append(fall_label)

        if len(sequence) == sequence_length:
            sequences.append(sequence)
            # Use the most frequent label in the sequence as the sequence label
            labels.append(max(set(sequence_labels), key=sequence_labels.count))

    return np.array(sequences), np.array(labels)

# Example usage
sequence_length = 30
sequences, labels = create_sequences("/Users/varunshankarhoskere/Desktop/Academics/PES/Capstone/Datasets/images", 
                                     "/Users/varunshankarhoskere/Desktop/Academics/PES/Capstone/Datasets/labels", 
                                     sequence_length)


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert sequences and labels to PyTorch tensors
sequences_tensor = torch.tensor(sequences, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Create a dataset and data loader
dataset = TensorDataset(sequences_tensor, labels_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# LSTM model initialization and training loop
# (Use the FallDetectionLSTM class and training loop provided earlier)
